In [1]:
import json
full_coco = json.load(open('/home/jess/ct_classifier_wd/data/intermediate/cls_coco.json'))

In [4]:
full_coco['images'][0]

{'file_name': 'Red Kangaroo/PCAM01_2021-05-24_17-58-33_M_1_3_7.JPG',
 'height': 254,
 'width': 349,
 'id': 'PCAM01_2021-05-24_17-58-33_M_1_3'}

In [3]:
full_coco['annotations'][0]

{'id': '8697519e-3c93-11ee-bfaa-22169906f295',
 'image_id': 'PCAM01_2021-05-24_17-58-33_M_1_3',
 'category_id': 0,
 'iscrowd': 0,
 'segmentation': [],
 'area': 88646}

In [5]:
full_coco['categories']

[{'id': 14, 'name': 'Goat'}]